In [1]:
import sys
sys.path.append('../../python_scripts')
import pymesh
import numpy as np
import logging, sys
logging.disable(sys.maxsize)
dirname = "/home/jxt/PPPM-TDBEM/dataset/bowl"
from fem import FEMmodel, Material, MatSet, LOBPCG_solver, SoundObj
from visulize import viewer
obj = SoundObj(dirname)

original mesh data
vertices:  (18370, 3)
faces:  (36736, 3)
bbox_min:  [-0.07172556 -0.03272797 -0.07172596]
bbox_max:  [0.07172556 0.02840117 0.0717259 ]
target cell_size:  0.0047817284295000005


In [2]:
obj.tetrahedralize()
obj.modal_analysis(k=200, material=Material(MatSet.Plastic))

tetrahedralized mesh data
vertices:  (3943, 3)
faces:  (7262, 3)
voxels:  (12191, 4)


In [3]:
obj.show_frequencys(num=20)
obj.visualize_modes(viewer, num=200)

[ 627.3537  628.4687 1305.9037 1311.5353 2175.1426 2181.4602 2403.6987
 2590.6313 2593.4993 3097.884  3203.1104 3233.9067 3534.3472 3540.438
 3819.6062 3823.664  4049.089  4058.9707 4239.397  4244.2334]


Box(children=(Box(children=(IntSlider(value=0, description='Mode Index', layout=Layout(flex='3 1 auto'), max=1…

In [4]:
vertices = obj.tet_mesh.vertices
tets = obj.tet_mesh.voxels
print(vertices.shape)
print(tets.shape)
print(obj.eigenvalues.shape)
print(obj.eigenvectors.shape)
np.savetxt(dirname + "/vertices.txt", vertices)
np.savetxt(dirname + "/tets.txt", tets.astype(int))
np.savetxt(dirname + "/eigenvalues.txt", obj.eigenvalues)
np.savetxt(dirname + "/eigenvectors.txt", obj.eigenvectors)

(3943, 3)
(12191, 4)
(200,)
(11829, 200)


In [5]:
from physical import PhysicalAnimation
import pybullet as p
anim = PhysicalAnimation(dirname)
anim.generate()
anim.post_process_contacts()

pybullet build time: May 20 2022 19:43:01
EGL device choice: -1 of 3.


Loaded EGL 1.5 after reload.
GL_VENDOR=NVIDIA Corporation
GL_RENDERER=NVIDIA GeForce RTX 3080 Ti/PCIe/SSE2
GL_VERSION=4.6.0 NVIDIA 515.65.01
GL_SHADING_LANGUAGE_VERSION=4.60 NVIDIA
Version = 4.6.0 NVIDIA 515.65.01
Vendor = NVIDIA Corporation
Renderer = NVIDIA GeForce RTX 3080 Ti/PCIe/SSE2
Generating animation...


  1%|          | 168/30000 [00:00<00:28, 1064.59it/s]

ven = NVIDIA Corporation
ven = NVIDIA Corporation


100%|██████████| 30000/30000 [00:05<00:00, 5904.95it/s]


post processing contact forces....


100%|██████████| 30000/30000 [00:02<00:00, 14976.39it/s]


In [8]:
SR = 44100
from tqdm import tqdm
import scipy.signal
from audio import save_audio, show_audio, IIR
sound_signal = np.zeros(int(anim.time_length * SR))
f = np.zeros((len(obj.eigenvalues), len(sound_signal)))
for contact in tqdm(anim.contact):
    t = int(contact[0] * SR)
    vertex_id = int(contact[1])
    force_direction = contact[2:5]
    impulse = contact[5] * (obj.eigenvectors[vertex_id*3, :] * force_direction[0] +
                             obj.eigenvectors[vertex_id*3+1, :] * force_direction[1] +
                             obj.eigenvectors[vertex_id*3+2, :] * force_direction[2])
    f[:, t] += impulse
# smooth force 
filter_width = 21
filter = scipy.signal.savgol_coeffs(filter_width, 3)
f = scipy.signal.savgol_filter(f, filter_width, 3, axis=1)
np.savetxt(dirname + "/filter.txt", filter)

for mode_idx in range(200):
    sound_signal += IIR(f[mode_idx], obj.eigenvalues[mode_idx], obj.fem_model.material.alpha, obj.fem_model.material.beta, 1/SR)

# get acceleration
sound_signal = np.diff(sound_signal, n=2)
save_audio(sound_signal, SR, dirname + "/sound.wav")

100%|██████████| 41349/41349 [00:00<00:00, 63779.73it/s]


In [10]:
import os
from IPython.display import Video
video_name = dirname + '/animation.mp4'
audio_name = dirname + '/sound.wav'
output_name = dirname + '/animation+sound.mp4'
os.system(f"ffmpeg -hide_banner -loglevel error -i {video_name} -i {audio_name} -c:v copy -c:a libvorbis -f mp4 {output_name} -y")
Video(output_name, embed=True)